In [12]:
import os
import string
import json
import cv2
import numpy as np

In [14]:
_DATA_DIRNAME = '../../handwriting_recognition/data/iam'
!ls {_DATA_DIRNAME}

forms	      formsI-Z.tgz	     LWITLR_task    resized_words_5
formsA-D.tgz  label_by_line_id.json  resized_lines  words_5
formsE-H.tgz  lines		     resized_words  xml


In [15]:
def get_reverse_dict(input_dict):
    return {v: k for k, v in input_dict.items()}

def read_image(image_uri, grayscale=True):
    """
    Read image_uri.
    """
    def read_image_from_filename(image_filename, grayscale):
        imread_flag = cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR
        return cv2.imread(image_filename, imread_flag)

    def read_image_from_url(image_url, grayscale):
        url_response = urlopen(image_url)
        img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
        imread_flag = cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR
        return cv2.imdecode(img_array, imread_flag)

    local_file = os.path.exists(image_uri)
    try:
        img = None
        if local_file:
            img = read_image_from_filename(image_uri, grayscale)
        else:
            img = read_image_from_url(image_uri, grayscale)
    except Exception as e:
        raise ValueError("Could not load image at {}: {}".format(image_uri, e))
    return img

def resize(image, scale_factor):
    """
    Resize image.
    OpenCV does not resize in place, hence need to copy and return in case
    scale_factor is 1 to maintain consistency in behaviour
    """
    if scale_factor == 1:
        return image.copy()
    return cv2.resize(
        image,
        dsize=None,
        fx=scale_factor,
        fy=scale_factor,
        interpolation=cv2.INTER_AREA
    )

In [16]:
def char_to_int_mapping():
    """
    Creates and returns a hash table mapping the characters to integer class indexes.
    The characters are an extension of IAM graphemes.
        00 -- 09: Digits
        10 -- 35: Capital Letters
        36 -- 61: Lower Letters
        62 -- 76: - , ; : ! ? / . ' " ( ) * & +
        77: # (garbage symbol)
        78: word separator (space in ground truth)
        79 -- 92: = _ [ ] { } ^ $ > < % | @ ~
        93: <ctc blank label> - not a part of the dict
    """
    return {
        char: ind for ind, char
        in enumerate(
            string.digits +
            string.ascii_uppercase +
            string.ascii_lowercase +
            '-,;:!?/.\'"()*&+# =_[]{}^$><%|@~'
        )
    }

char_to_int_dict = char_to_int_mapping()

In [20]:
def get_iam_labels(label_file):
    with open(label_file, 'r') as f:
        labels_dict = json.load(f)
    return labels_dict


def get_line_id_from_filename(filename):
    """
    Input example: '../data/iam/words_5/g06-031e-02-00.png' or '../data/iam/words_5/g06-031e-02.png'
    Output example: 'g06-031e-02'  (line_id)
    """
    return '-'.join(os.path.splitext(os.path.basename(filename))[0].split('-')[:3])


def iam_helper(image_directory, label_dict, label_to_int_mapping, image_height):
    """Create an IAM dataset object from IAM-DB images
    Args:
        buckets: a sorted list of integers describing image widths of all buckets
        label_dict: a dictionary with key being image id and value being its label
    """
    dirname = os.path.join(_DATA_DIRNAME, 'LWITLR_task')
    train_index_path = os.path.join(dirname, 'trainset.txt')
    val_index_path = os.path.join(dirname, 'validationset2.txt')
    test_index_path = os.path.join(dirname, 'testset.txt')

    assert os.path.exists(train_index_path) and os.path.exists(val_index_path) and os.path.exists(test_index_path)

    int_to_label_mapping = get_reverse_dict(label_to_int_mapping)

    def load_iam_lines(index_path):
        with open(index_path, 'r') as f:
            image_filenames = f.read().splitlines()
        image_filenames = [os.path.join(image_directory, '{}.png'.format(fname)) for fname in image_filenames]
        return load(image_filenames, get_line_id_from_filename)

    def load_image(image_filename):
        image = read_image(image_filename, grayscale=True)
        #image = 255. - image
        image = resize(image, (image_height / image.shape[0]))
        return image / 255.
        
    def load(image_filenames, get_label_key_func):
        images = [load_image(fname) for fname in image_filenames]
        labels = [label_dict[get_label_key_func(filename)] for filename in image_filenames]
        int_labels = [[label_to_int_mapping[c] for c in label] for label in labels]
        return images, labels,  #int_labels

    x_train, y_train = load_iam_lines(train_index_path), 
    x_val, y_val = load_iam_lines(val_index_path), 
    x_test, y_test = load_iam_lines(test_index_path)

    print(f'Number of images in train: {len(x_train)}')
    print(f'Number of images in val: {len(x_val)}')
    print(f'Number of images in test: {len(x_test)}')

    return x_train, y_train, x_val, y_val, x_test, y_test

In [19]:
image_height = 28
image_directory = os.path.join(_DATA_DIRNAME, 'lines')
label_by_line_id = get_iam_labels(os.path.join(_DATA_DIRNAME, 'label_by_line_id.json'))

assert os.path.exists(image_directory)

x_train, y_train, x_val, y_val, x_test, y_test = iam_helper(
    image_directory=image_directory,
    label_dict=label_by_line_id,
    label_to_int_mapping=char_to_int_dict,
    image_height=image_height
)

ValueError: not enough values to unpack (expected 2, got 1)